In [1]:
import pandas as pd
from geopy import distance
GARDINER_LOCS_MIDDLE_POINT = (43.638933, -79.457196)

If loc lng > loc middle point lng, loc is east of point (+)
If loc lng < loc middle point lng, loc is west of point (-)

In [2]:
df = pd.read_csv("../../csv/v2/040_gardiner_count_processed.csv.zip")
df.head(3)

,detector_id,datetime_bin,volume_15min,number_of_lanes,primary_road,direction,latitude,longitude,detector_loc
0,DW0040DWG,2022-08-11 15:00:00,160,3,F G Gardiner Xy W,W,43.638046,-79.396634,W of SPADINA AV
1,DW0126DEG,2022-08-11 15:00:00,170,3,F G Gardiner Xy E,E,43.638420,-79.451353,W of RONCESVALLES AVE
2,DW0140DEG,2022-08-11 15:00:00,169,3,F G Gardiner Xy E,E,43.637296,-79.464739,E of ELLIS AVE


In [3]:
len(df)

145099

In [4]:
df.sort_values(by="datetime_bin",inplace=True)

In [5]:
df

,detector_id,datetime_bin,volume_15min,number_of_lanes,primary_road,direction,latitude,longitude,detector_loc
0,DW0040DWG,2022-08-11 15:00:00,160,3,F G Gardiner Xy W,W,43.638046,-79.396634,W of SPADINA AV
21,DW0080DEG,2022-08-11 15:00:00,121,3,F G Gardiner Xy E,E,43.634592,-79.422319,E of DUFFERIN STREET
20,DW0126DWG,2022-08-11 15:00:00,222,3,F G Gardiner Xy W,W,43.638420,-79.451353,E of PARKSIDE DRIVE
19,DW0151DEG,2022-08-11 15:00:00,179,4,F G Gardiner Xy E,E,43.631469,-79.475946,W of HUMBER RIVER (PALACE PIER)
18,DW0161DWG,2022-08-11 15:00:00,334,5,F G Gardiner Xy W,W,43.624484,-79.493290,W of GRAND AVE
...,...,...,...,...,...,...,...,...,...
145088,DW0161DWG,2023-01-01 05:15:00,538,5,F G Gardiner Xy W,W,43.624484,-79.493290,W of GRAND AVE
145087,DW0176DEG,2023-01-01 05:15:00,52,2,F G Gardiner C E,E,43.620000,-79.513960,W of RAMP ISLINGTON S TO FGG E
145097,DW0151DWG,2023-01-01 05:15:00,365,5,F G Gardiner Xy W,W,43.631070,-79.475995,W of HUMBER RIVER (PALACE PIER)
145091,DW0151DEG,2023-01-01 05:15:00,217,4,F G Gardiner Xy E,E,43.631469,-79.475946,W of HUMBER RIVER (PALACE PIER)


# Expand datetime_bin

In [6]:
df["datetime_bin"] = pd.to_datetime(df["datetime_bin"],utc=True)
df["year"] = df["datetime_bin"].dt.year
df["month"] = df["datetime_bin"].dt.month_name()
df["day"] = df["datetime_bin"].dt.day
df["day_of_week"] = df["datetime_bin"].dt.day_name()
df["hour"] = df["datetime_bin"].dt.hour
df["minute"] = df["datetime_bin"].dt.minute
df.head(3)

,detector_id,datetime_bin,volume_15min,number_of_lanes,primary_road,direction,latitude,longitude,detector_loc,year,month,day,day_of_week,hour,minute
0,DW0040DWG,2022-08-11 15:00:00+00:00,160,3,F G Gardiner Xy W,W,43.638046,-79.396634,W of SPADINA AV,2022,August,11,Thursday,15,0
21,DW0080DEG,2022-08-11 15:00:00+00:00,121,3,F G Gardiner Xy E,E,43.634592,-79.422319,E of DUFFERIN STREET,2022,August,11,Thursday,15,0
20,DW0126DWG,2022-08-11 15:00:00+00:00,222,3,F G Gardiner Xy W,W,43.638420,-79.451353,E of PARKSIDE DRIVE,2022,August,11,Thursday,15,0


# Understand categorical features

## Detector ID and detector loc

In [7]:
len(df["detector_id"].drop_duplicates())

29

In [8]:
df["detector_id"].value_counts()

detector_id
DW0151DWG    7058
DW0146DEG    7058
DW0126DWG    7058
DW0151DEG    7058
DW0146DWG    7058
DW0126DEG    7058
DW0161DWG    7045
DW0161DEG    7045
DW0040DEG    7007
DW0040DWG    7007
DW0201DEG    6982
DW0201DWG    6982
DW0110DEG    6784
DW0080DEG    6607
DW0100DWG    6593
DW0100DEG    6593
DW0140DWG    6465
DW0140DEG    6463
DW0090DWG    4408
DW0090DEG    4408
DW0176DEG    3574
DW0130DEG    2934
DW0070DWG    1934
DW0070DEG    1933
DW0060DWG     564
DW0060DEG     564
DE0020DEG     376
DE0020DWG     376
DE0060DWG     107
Name: count, dtype: int64

In [9]:
len(df["detector_loc"].drop_duplicates())

23

In [10]:
df["detector_loc"].value_counts()

detector_loc
W of HUMBER RIVER  (PALACE PIER)    14116
E of WINDERMERE AVE                 14116
W of GRAND AVE                      14090
At THE EAST MALL                    13964
E of ELLIS AVE                      12928
E of DUNN AV                         8816
W of RONCESVALLES AVE                7058
E of PARKSIDE DRIVE                  7058
W of SPADINA AV                      7007
W of SPADINA AVE                     7007
W of DOWLING AV                      6784
E of DUFFERIN STREET                 6607
W of JAMESON AVE                     6593
W of JAMESON AV                      6593
W of RAMP ISLINGTON S TO FGG E       3574
E of COLBOURNE LODGE DRIVE           2934
E of NOVA SCOTIA AV                  1934
W of GARRISON ROAD                   1933
E of STRACHAN AV                      564
E of STRACHAN AVE                     564
E of JARVIS STREET                    376
W of JARVIS STREET                    376
E of DON VALLEY PKWY                  107
Name: count, dtype: i

In [11]:
df.drop_duplicates("detector_id")

,detector_id,datetime_bin,volume_15min,number_of_lanes,primary_road,direction,latitude,longitude,detector_loc,year,month,day,day_of_week,hour,minute
0,DW0040DWG,2022-08-11 15:00:00+00:00,160,3,F G Gardiner Xy W,W,43.638046,-79.396634,W of SPADINA AV,2022,August,11,Thursday,15,0
21,DW0080DEG,2022-08-11 15:00:00+00:00,121,3,F G Gardiner Xy E,E,43.634592,-79.422319,E of DUFFERIN STREET,2022,August,11,Thursday,15,0
20,DW0126DWG,2022-08-11 15:00:00+00:00,222,3,F G Gardiner Xy W,W,43.638420,-79.451353,E of PARKSIDE DRIVE,2022,August,11,Thursday,15,0
19,DW0151DEG,2022-08-11 15:00:00+00:00,179,4,F G Gardiner Xy E,E,43.631469,-79.475946,W of HUMBER RIVER (PALACE PIER),2022,August,11,Thursday,15,0
18,DW0161DWG,2022-08-11 15:00:00+00:00,334,5,F G Gardiner Xy W,W,43.624484,-79.493290,W of GRAND AVE,2022,August,11,Thursday,15,0
17,DW0161DEG,2022-08-11 15:00:00+00:00,234,4,F G Gardiner Xy E,E,43.624484,-79.493290,W of GRAND AVE,2022,August,11,Thursday,15,0
16,DW0090DEG,2022-08-11 15:00:00+00:00,155,3,F G Gardiner Xy E,E,43.632581,-79.429757,E of DUNN AV,2022,August,11,Thursday,15,0
14,DW0060DWG,2022-08-11 15:00:00+00:00,212,3,F G Gardiner Xy W,W,43.637581,-79.409871,E of STRACHAN AV,2022,August,11,Thursday,15,0
13,DW0151DWG,2022-08-11 15:00:00+00:00,260,5,F G Gardiner Xy W,W,43.631070,-79.475995,W of HUMBER RIVER (PALACE PIER),2022,August,11,Thursday,15,0
12,DW0090DWG,2022-08-11 15:00:00+00:00,215,3,F G Gardiner Xy W,W,43.632725,-79.429807,E of DUNN AV,2022,August,11,Thursday,15,0


- I will work with detector ID because there are some location names with two IDs
    - This means I am dropping detector_loc

In [12]:
df = df.drop("detector_loc",axis=1)
df.head(3)

,detector_id,datetime_bin,volume_15min,number_of_lanes,primary_road,direction,latitude,longitude,year,month,day,day_of_week,hour,minute
0,DW0040DWG,2022-08-11 15:00:00+00:00,160,3,F G Gardiner Xy W,W,43.638046,-79.396634,2022,August,11,Thursday,15,0
21,DW0080DEG,2022-08-11 15:00:00+00:00,121,3,F G Gardiner Xy E,E,43.634592,-79.422319,2022,August,11,Thursday,15,0
20,DW0126DWG,2022-08-11 15:00:00+00:00,222,3,F G Gardiner Xy W,W,43.638420,-79.451353,2022,August,11,Thursday,15,0


## Primary Road
- This categorical feature just gives the name of the road, as well as travel direction, which we already have
    - This means I can drop primary_road

In [13]:
df = df.drop("primary_road",axis=1)
df.head(3)

,detector_id,datetime_bin,volume_15min,number_of_lanes,direction,latitude,longitude,year,month,day,day_of_week,hour,minute
0,DW0040DWG,2022-08-11 15:00:00+00:00,160,3,W,43.638046,-79.396634,2022,August,11,Thursday,15,0
21,DW0080DEG,2022-08-11 15:00:00+00:00,121,3,E,43.634592,-79.422319,2022,August,11,Thursday,15,0
20,DW0126DWG,2022-08-11 15:00:00+00:00,222,3,W,43.638420,-79.451353,2022,August,11,Thursday,15,0


## Detector ID
- Can be dropped because onehot encoding this would create lots of columns

In [14]:
df = df.drop("detector_id",axis=1)
df.head(3)

,datetime_bin,volume_15min,number_of_lanes,direction,latitude,longitude,year,month,day,day_of_week,hour,minute
0,2022-08-11 15:00:00+00:00,160,3,W,43.638046,-79.396634,2022,August,11,Thursday,15,0
21,2022-08-11 15:00:00+00:00,121,3,E,43.634592,-79.422319,2022,August,11,Thursday,15,0
20,2022-08-11 15:00:00+00:00,222,3,W,43.638420,-79.451353,2022,August,11,Thursday,15,0


# Understanding numerical features

## Latitude and longitude
- These attributes will be useful
    - By determining the distance from a middle point

In [15]:
def calculate_distance(coord):
    if GARDINER_LOCS_MIDDLE_POINT[1] > coord[1]:
        return -distance.distance(GARDINER_LOCS_MIDDLE_POINT,coord).km
    return distance.distance(GARDINER_LOCS_MIDDLE_POINT,coord).km

df["dist_from_mid_point"] = df.apply(lambda x: calculate_distance((x["latitude"],x["longitude"])),axis=1)
df.head(3)

,datetime_bin,volume_15min,number_of_lanes,direction,latitude,longitude,year,month,day,day_of_week,hour,minute,dist_from_mid_point
0,2022-08-11 15:00:00+00:00,160,3,W,43.638046,-79.396634,2022,August,11,Thursday,15,0,4.887837
21,2022-08-11 15:00:00+00:00,121,3,E,43.634592,-79.422319,2022,August,11,Thursday,15,0,2.855389
20,2022-08-11 15:00:00+00:00,222,3,W,43.638420,-79.451353,2022,August,11,Thursday,15,0,0.474912


- Now I can drop lat and lng because they were already used to calculate the distance from the middle point

In [16]:
df = df.drop(["latitude","longitude"],axis=1)
df.head(3)

,datetime_bin,volume_15min,number_of_lanes,direction,year,month,day,day_of_week,hour,minute,dist_from_mid_point
0,2022-08-11 15:00:00+00:00,160,3,W,2022,August,11,Thursday,15,0,4.887837
21,2022-08-11 15:00:00+00:00,121,3,E,2022,August,11,Thursday,15,0,2.855389
20,2022-08-11 15:00:00+00:00,222,3,W,2022,August,11,Thursday,15,0,0.474912


In [17]:
df["number_of_lanes"].value_counts()

number_of_lanes
3    112836
4     14479
5     14103
2      3681
Name: count, dtype: int64

- Drop year because the data is seasonal

In [18]:
df = df.drop("year",axis=1)
df.head(3)

,datetime_bin,volume_15min,number_of_lanes,direction,month,day,day_of_week,hour,minute,dist_from_mid_point
0,2022-08-11 15:00:00+00:00,160,3,W,August,11,Thursday,15,0,4.887837
21,2022-08-11 15:00:00+00:00,121,3,E,August,11,Thursday,15,0,2.855389
20,2022-08-11 15:00:00+00:00,222,3,W,August,11,Thursday,15,0,0.474912


In [20]:
min_vol = df["volume_15min"].min()
max_vol = df["volume_15min"].max()
mean_vol = df["volume_15min"].mean()
median_vol = df["volume_15min"].median()
variance = df["volume_15min"].var()
std_dev = df["volume_15min"].std()

print(
    f"Min volume:    {min_vol}\n"
    f"Max volume:    {max_vol}\n"
    f"Mean volume:   {mean_vol}\n"
    f"Median volume: {median_vol}\n"
    f"Variance:     {variance}\n"
    f"Std dev:      {std_dev}"
)

Min volume:    0
Max volume:    1904
Mean volume:   735.7366625545318
Median volume: 782.0
Variance:     154462.27169138243
Std dev:      393.01688474082437


# What are the features and target of this dataset?
- Target: volume_15min
- Features: number_of_lanes, year, month, day, day_of_week, hour, minute, distance

In [17]:
df.to_csv("./data/gardiner_count.csv.zip",index=False, compression="zip")